In [1]:
import sys
sys.path.append("../..")
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import random
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch import distributions
from torch.nn.parameter import Parameter
from train import train
from models.models_new import Renorm_Dynamic
from dynamic_models_sis_new import Simple_Spring_Model
from models_new import VAE
from train import train_vae
from ei.EI_calculation import test_vae_causal_multi_sis

from datetime import datetime
t0 = datetime.now()


use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0') if use_cuda else torch.device('cpu')
device

device(type='cuda', index=0)

# Generate data

In [3]:
# whole sample
mul_batch_size = [0,5000,3000,1000]#[0,10,4500,1500]
sigma = 0.03
rou = -0.5
steps = 7
dt = 1
interval = 1
seed = 2050
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
spring = Simple_Spring_Model(device=device)
test_data = spring.generate_multistep_sir(size_list=[500,500], steps=10*interval, sigma=sigma, rou=rou,lam=1,miu=0.5,dt=dt,interval=interval) 
train_data = spring.generate_multistep_sir(size_list=mul_batch_size, steps=steps*interval, sigma=sigma,rou=rou,lam=1,miu=0.5,dt=dt,interval=interval)

# Five parallel experiments

In [ ]:
sz = 4
scale = 2
mae2_w = 3
T_total = 30001
experiments = 5
MAEs_mstep_e = np.zeros([experiments, T_total//500+1])
eis_e = np.zeros([experiments, T_total//500+1])
for e in range(experiments):
    seed = 1 + e
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    test_data = spring.generate_multistep_sir(size_list=[500,500], steps=10*interval, sigma=sigma, rou=rou,lam=1,miu=0.5,dt=dt,interval=interval) 
    train_data = spring.generate_multistep_sir(size_list=mul_batch_size, steps=steps*interval, sigma=sigma,rou=rou,lam=1,miu=0.5,dt=dt,interval=interval)
    eis_vae, term1s_vae, term2s_vae, losses_vae, MAEs_mstep_vae, vae = train_vae(train_data=train_data, test_data=test_data, 
                                                            sz=sz, scale=scale, mae2_w=mae2_w, T2=T_total,  sigma=0.03, rou=-0.5, dt=dt, 
                                                                             L=1, hidden_units = 64, batch_size = 700)
    MAEs_mstep_e[e,:] = MAEs_mstep_vae
    eis_e[e,:] = eis_vae

check point-------o_0-> 14:52:35.800423; lasting 0:00:04.192397 seconds
Epoch: 0
Train loss: 0.3949
dEI: 0.5566
term1: -0.3990
term2: 1.5121
Test multistep loss: 0.4480
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 14:52:46.737209; lasting 0:00:10.936786 seconds
check point-------o_0-> 14:52:53.411371; lasting 0:00:06.674162 seconds
Epoch: 500
Train loss: 0.0669
dEI: 0.0000
term1: -2.9074
term2: 1.8512
Test multistep loss: 0.0578
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 14:52:58.850593; lasting 0:00:05.439222 seconds
check point-------o_0-> 14:53:05.476559; lasting 0:00:06.625966 seconds
Epoch: 1000
Train loss: 0.0626
dEI: 0.0000
term1: -3.3434
term2: 1.9988
Test multistep loss: 0.0610
------------------------------------------------------------------------------------------------------

# Train

In [ ]:
sz = 4
scale = 2
mae2_w = 3
T_total = 30001

eis_vae, term1s_vae, term2s_vae, losses_vae, MAEs_mstep_vae, vae = train_vae(train_data=train_data, test_data=test_data, 
                                                            sz=sz, scale=scale, mae2_w=mae2_w, T2=T_total,  sigma=0.03, rou=-0.5, dt=dt, 
                                                                             L=1, hidden_units = 64, batch_size = 700)